In [1]:
import sys
import os
import numpy as np
import json
import subprocess
import time

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
sys.path.append(os.path.expanduser('~/git/analysis_playground/bayesian-fitting'))
import importlib
import model_utils
importlib.reload(model_utils)

<module 'model_utils' from '/SNS/users/m2d/git/analysis_playground/bayesian-fitting/model_utils.py'>

In [3]:
project_dir = os.path.expanduser('~/git/nh3-analysis/jan2023/')

dyn_model_dir = os.path.expanduser(os.path.join(project_dir, 'dyn-fitting'))
dyn_data_dir = '/SNS/REF_L/IPTS-29196/shared/dynamic/Cu-B-Expt-6/15s'
results_dir = os.path.expanduser(os.path.join(project_dir, 'dyn-fitting'))


# Auto-reduction directory
ar_dir = '/SNS/REF_L/IPTS-29196/shared/autoreduce/'


dyn_model_dir = os.path.expanduser(os.path.join(project_dir, 'dyn-fitting'))


# Initial data set and model (starting point)
initial_data_file = os.path.join(ar_dir, 'REFL_201290_combined_data_auto.txt')
initial_data = np.loadtxt(initial_data_file).T

initial_err_file = os.path.join(dyn_model_dir, '201290-err.json')
initial_expt_file = os.path.join(dyn_model_dir, '201290-expt.json')

#store_basename = 'results-5s-fwd'
store_basename = '201289-dyn/results-15s'

# Actual dynamic run
dynamic_run = 201289

In [6]:
PROCESS_ALL_DATA = False

first = 5
last = 60

if PROCESS_ALL_DATA:
    _file_list = sorted(os.listdir(dyn_data_dir))

    # Get only the files for the run we're interested in
    _good_files = [_f for _f in _file_list if _f.startswith('r%d_t' % dynamic_run)]
    _good_files = _good_files[first:last]
    _good_files.reverse()
else:
    _good_files = []
    for t in [8010, 1200, 900, 600, 495, 405, 300, 225, 210, 195, 180, 165, 150]:
        _good_files.append('r{0}_t{1:06d}.txt'.format(dynamic_run, t))


starting_model = initial_expt_file
starting_err = initial_err_file

with open(initial_err_file, 'r') as fd:
    initial_model = json.load(fd)
    
time_series = [initial_model]

t0 = time.time()
t1 = time.time()
for _file in _good_files:
    print(_file)
    _base_name, _ = os.path.splitext(_file)
    data_to_fit = os.path.join(dyn_data_dir, _file)
    command = ['refl1d_cli.py', '--fit=dream', '--steps=1000', '--burn=1000', '--batch', '--overwrite',
               '--store=%s' % os.path.join(results_dir, store_basename, _base_name), 
               os.path.join(project_dir, 'model-loop-v2.py'),
               data_to_fit, starting_model, starting_err]

    out = subprocess.run(command, capture_output=False)

    # Update the starting model with the fit we just did
    _model = os.path.join(results_dir, store_basename, _base_name, 'model-loop-v2-expt.json')
    _err = os.path.join(results_dir, store_basename, _base_name, 'model-loop-v2-err.json')
    starting_model = _model
    starting_err = _err
    
    print(starting_model)
    
    with open(os.path.join(_err), 'r') as fd:
        updated_model = json.load(fd)
        time_series.append(updated_model)

    model_utils.print_model(time_series[-2], time_series[-1])

    total_time = (time.time()-t0)/30.0
    item_time = time.time()-t1
    t1 = time.time()
    print("    Completed: %g s [total=%g m]" % (item_time, total_time))

r201289_t008010.txt
/SNS/users/m2d/git/nh3-analysis/jan2023/dyn-fitting/201289-dyn/results-15s/r201289_t008010/model-loop-v2-expt.json
                          Initial                 Step
SEI interface           13.1 +- 4.3           13.8 +- 3.4    
SEI rho                 3.17 +- 0.062         3.36 +- 0.16   
SEI thickness            240 +- 6.9            243 +- 4.6    
THF interface           73.4 +- 2.9           72.6 +- 2.7    
material interface      18.9 +- 0.74          19.6 +- 2.5    
material rho            2.74 +- 0.065         2.75 +- 0.12   
material thickness       101 +- 4.9            105 +- 4.2    
    Completed: 86.5731 s [total=2.88577 m]
r201289_t001200.txt
/SNS/users/m2d/git/nh3-analysis/jan2023/dyn-fitting/201289-dyn/results-15s/r201289_t001200/model-loop-v2-expt.json
                          Initial                 Step
SEI interface           13.8 +- 3.4           14.5 +- 3.2    
SEI rho                 3.36 +- 0.16          3.56 +- 0.18   
SEI thickness      